In [1]:
! pip install scikit-learn openTSNE torch_dct

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 65.3 MB/s eta 0:00:00a 0:00:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import re
import glob
import numpy as np
import importlib
import random
import copy
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils.common import load_config, setup_seed
from data.dataset import ImageDataset
from models.models import Simple_CNN
# from MulticoreTSNE import MulticoreTSNE as TSNE
from openTSNE import TSNE  # 直接导入 openTSNE 的 TSNE 类

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei', 'WenQuanYi Zen Hei']
plt.rcParams['axes.unicode_minus'] = False


def get_feature(model, dataloader, device):
    """提取模型对数据集的特征"""
    model.eval()
    features = []
    labels = []
    
    # 创建进度条
    total_batches = len(dataloader)
    progress_bar = tqdm(enumerate(dataloader), total=total_batches, desc="提取特征")
    
    with torch.no_grad():
        for i, batch in progress_bar:
            input_img_batch, label_batch, _ = batch 
            input_img = input_img_batch.reshape((-1, 3, input_img_batch.size(-2), input_img_batch.size(-1))).to(device)
            label = label_batch.reshape((-1)).to(device)
            
            # 获取模型特征
            _, feature = model(input_img, data='dct')
            
            if i == 0:
                features = feature
                gt_labels = label
            else:
                features = torch.cat([features, feature], dim=0)
                gt_labels = torch.cat([gt_labels, label])
            
            # 更新进度条信息
            progress_bar.set_postfix({"批次": i+1, "样本数": features.shape[0]})

    features = features.cpu().numpy()
    labels = gt_labels.cpu().numpy()
        
    return features, labels



def tsne_analyze(features, labels, class_names=None, title="t-SNE特征可视化"):
    """使用t-SNE对特征进行降维并可视化，返回主图、图例手柄和标签"""
    print(f">>> {title} - t-SNE 拟合开始")
    
    # 随机打乱数据
    indexs = list(range(len(features)))
    random.shuffle(indexs)
    features = features[indexs]
    labels = labels[indexs]
    
    # t-SNE降维
    embeddings = TSNE(
        n_jobs=4,
        verbose=True,
        callbacks=lambda it, error, emb: tqdm.write(f"t-SNE进度: {it}迭代, 误差: {error:.4f}"),
        callbacks_every_iters=10
    ).fit(features)
    
    print(f"<<< {title} - t-SNE 拟合完成")
    
    vis_x, vis_y = embeddings[:, 0], embeddings[:, 1]
    unique_labels = np.unique(labels)
    
    # 为不同类别设置颜色和标记
    markers = ['o', 's', '^', 'D', 'v', '*', 'p', 'h', '8', 'x']
    colors = plt.cm.tab20.colors
    
    # 初始化主图
    plt.figure(figsize=(10, 8))
    handles, legend_labels = [], []  # 收集图例的手柄和标签
    
    # 绘制每个类别（用tqdm显示进度）
    for i, label in enumerate(tqdm(unique_labels, desc="绘制类别")):
        class_index = np.where(labels == label)[0]
        marker_idx = i % len(markers)
        color_idx = i % len(colors)
        
        # 类别名称处理
        label_text = f"类别 {label}"
        if class_names and label < len(class_names):
            label_text = class_names[label]
        
        # 绘制散点并记录图例元素
        scatter = plt.scatter(
            vis_x[class_index], vis_y[class_index], 
            s=40, color=colors[color_idx], alpha=0.8, marker=markers[marker_idx]
        )
        handles.append(scatter)
        legend_labels.append(label_text)
    
    # 主图样式设置（无图例）
    plt.title(title, fontsize=16)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    print(f"<<< {title} - 主图绘制完成")
    
    return plt.gcf(), handles, legend_labels  # 返回主图、图例手柄、图例标签


def load_train_list(file_path, root_dir="/mnt/workspace/POSE"):
    """加载训练集列表文件"""
    samples = []
    class_set = set()
    
    print(f"加载训练集列表: {file_path}")
    
    # 先获取文件总行数用于进度显示
    total_lines = 0
    with open(file_path, 'r') as f:
        total_lines = sum(1 for _ in f)
    
    # 使用tqdm显示读取进度
    with open(file_path, 'r') as f:
        for line in tqdm(f, total=total_lines, desc="读取训练集列表"):
            line = line.strip()
            if not line:
                continue
                
            # 分割路径和标签
            parts = line.split('\t')
            if len(parts) < 2:
                continue
                
            img_path, label = parts[0], int(parts[1])
            
            # 构建完整路径
            if img_path.startswith('./'):
                img_path = os.path.join(root_dir, img_path[2:])
            else:
                img_path = os.path.join(root_dir, img_path)
                
            samples.append((img_path, label))
            class_set.add(label)
    
    print(f"共加载 {len(samples)} 个样本，{len(class_set)} 个类别")
    return samples, sorted(list(class_set))

def extract_epoch(filename):
    """从模型文件名中提取 epoch 数值"""
    match = re.search(r'model_(\d+)_test', os.path.basename(filename))
    return int(match.group(1)) if match else 0




def train_set_visualization(train_list_path, model_paths, output_dir=None):
    """训练集特征可视化主函数，支持多个模型，分离主图和图例"""
    # 加载训练集数据
    samples, classes = load_train_list(train_list_path)
    class_names = [f"类别 {i}" for i in classes]
    
    # 创建数据加载器
    train_set = ImageDataset(samples, config, balance=False, test_mode=True)
    train_loader = DataLoader(
        train_set, batch_size=config.batch_size, 
        num_workers=config.num_workers, pin_memory=True, shuffle=False
    )
    print(f"数据加载器创建完成，共 {len(train_loader)} 个批次")
    
    # 逐个模型处理
    for model_path in model_paths:
        epoch = extract_epoch(model_path)
        model_title = f"Epoch {epoch} 模型特征可视化"
        print(f"\n=== 处理模型: {model_path}（{model_title}）===")
        
        # 加载模型
        model = Simple_CNN(class_num=11, out_feature_result=True)
        pretrained_dict = torch.load(model_path, map_location='cpu')['state_dict']
        model.load_state_dict(pretrained_dict)
        model = model.to(device)
        
        # 提取特征
        features, labels = get_feature(model, train_loader, device)
        
        # 生成主图 + 图例元素
        main_fig, handles, legend_labels = tsne_analyze(features, labels, class_names, model_title)
        
        # 单独绘制图例图
        legend_fig = plt.figure(figsize=(4, 6))  # 调整尺寸适配图例数量
        legend_ax = legend_fig.add_subplot(111)
        legend_ax.legend(
            handles, legend_labels, 
            loc='center', fontsize=12, ncol=1  # ncol控制列数，可根据类别数量调整
        )
        legend_ax.axis('off')  # 隐藏坐标轴
        plt.tight_layout()
        
        # 保存图像（若指定输出目录）
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
            # 保存主图
            main_path = os.path.join(output_dir, f"main_epoch_{epoch}.png")
            main_fig.savefig(main_path, dpi=300, bbox_inches='tight')
            print(f"主图保存至: {main_path}")
            
            # 保存图例图
            legend_path = os.path.join(output_dir, f"legend_epoch_{epoch}.png")
            legend_fig.savefig(legend_path, dpi=300, bbox_inches='tight')
            print(f"图例图保存至: {legend_path}")
        
        # 关闭画布释放内存（可选）
        plt.close(main_fig)
        plt.close(legend_fig)
        
        

if __name__ == "__main__":
    # 设置设备
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    setup_seed(3)
    config = load_config('configs.{}'.format('progressive'))

    # 模型目录
    model_dir = '/mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0'
    
    # 获取所有模型文件并按epoch排序
    print(f"查找模型文件: {os.path.join(model_dir, 'model*.pth')}")
    pretrain_model_paths = glob.glob(os.path.join(model_dir, "model*.pth"))
    
    if not pretrain_model_paths:
        print(f"错误: 在 {model_dir} 目录下未找到模型文件")
        exit(1)
    
    # 按 epoch 从小到大排序
    pretrain_model_paths.sort(key=extract_epoch)
    
    print(f"找到 {len(pretrain_model_paths)} 个模型文件:")
    for i, path in enumerate(pretrain_model_paths):
        epoch = extract_epoch(path)
        print(f"  {i+1}. Epoch {epoch}: {path}")
    
    # 训练集列表文件路径
    train_list_path = "/mnt/workspace/POSE/dataset/annotations/train_list.txt"
    
    # 输出目录
    output_dir = "./output/multi_epoch_visualizations"
    
    # 执行训练集特征可视化
    train_set_visualization(train_list_path, pretrain_model_paths, output_dir)

查找模型文件: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model*.pth
找到 6 个模型文件:
  1. Epoch 4: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_4_test87.07000000000001_acc_AUC_78.23_OSCR_71.4817665289255.pth
  2. Epoch 9: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_9_test90.60000000000001_acc_AUC_80.77_OSCR_76.17168698347233.pth
  3. Epoch 14: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_14_test90.68_acc_AUC_80.09_OSCR_75.79871425619807.pth
  4. Epoch 19: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_19_test91.29_acc_AUC_80.77_OSCR_76.78650309917349.pth
  5. Epoch 24: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_24_test91.75_acc_AUC_80.8_OSCR_77.09646012396702.pth
  6. Epoch 29: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_29_test91.55_acc_AUC_79.76_OSCR_76.0735002066103.pth
加载训练集列表: /mnt/workspace/POSE/dataset/annotations/train_list.t

读取训练集列表: 100%|██████████| 44000/44000 [00:00<00:00, 606788.88it/s]

共加载 44000 个样本，11 个类别
数据加载器创建完成，共 5500 个批次

=== 处理模型: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_4_test87.07000000000001_acc_AUC_78.23_OSCR_71.4817665289255.pth（Epoch 4 模型特征可视化）===



提取特征: 100%|██████████| 5500/5500 [01:58<00:00, 46.25it/s, 批次=5500, 样本数=44000] 


>>> Epoch 4 模型特征可视化 - t-SNE 拟合开始
--------------------------------------------------------------------------------
TSNE(callbacks=<function tsne_analyze.<locals>.<lambda> at 0x7ff78d781e40>,
     callbacks_every_iters=10, early_exaggeration=12, n_jobs=4, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 21.18 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.53 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.11 seconds
===> Running optimization with exaggeration=12.00, lr=3666.67 for 250 iterations...
t-SNE进度: 10迭代, 误差: 6.9765
t-SNE进度: 20迭代, 误差: 6.9573
t-SNE进度: 30迭代, 误差: 6.1853
t-SNE进度: 40迭代, 误差: 5.3968
t-SNE进度: 50迭代, 误差: 5.1985
Iteration   50, KL divergence 5.1985, 50 iterations in 1.7803 sec
t-SNE进度: 60迭代, 误差: 5.1458
t-SNE进度: 70迭代, 误差: 5.1162
t-SNE进度: 80迭代, 误差: 5.0926
t-SNE进度: 90迭代, 误

...

<<< Epoch 24 模型特征可视化 - 主图绘制完成
主图保存至: ./output/multi_epoch_visualizations/main_epoch_24.png
图例图保存至: ./output/multi_epoch_visualizations/legend_epoch_24.png

=== 处理模型: /mnt/workspace/POSE/dataset/models/progressive/0612/POSE_seed0/model_29_test91.55_acc_AUC_79.76_OSCR_76.0735002066103.pth（Epoch 29 模型特征可视化）===


提取特征: 100%|██████████| 5500/5500 [01:58<00:00, 46.36it/s, 批次=5500, 样本数=44000] 


>>> Epoch 29 模型特征可视化 - t-SNE 拟合开始
--------------------------------------------------------------------------------
TSNE(callbacks=<function tsne_analyze.<locals>.<lambda> at 0x7ff785eaa5c0>,
     callbacks_every_iters=10, early_exaggeration=12, n_jobs=4, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 20.53 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.53 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.15 seconds
===> Running optimization with exaggeration=12.00, lr=3666.67 for 250 iterations...
t-SNE进度: 10迭代, 误差: 6.9525
t-SNE进度: 20迭代, 误差: 6.9374
t-SNE进度: 30迭代, 误差: 6.2135
t-SNE进度: 40迭代, 误差: 5.3308
t-SNE进度: 50迭代, 误差: 5.0998
Iteration   50, KL divergence 5.0998, 50 iterations in 2.0362 sec
t-SNE进度: 60迭代, 误差: 5.0208
t-SNE进度: 70迭代, 误差: 4.9674
t-SNE进度: 80迭代, 误差: 4.9373
t-SNE进度: 90迭代, 

绘制类别: 100%|██████████| 11/11 [00:00<00:00, 455.83it/s]


<<< Epoch 29 模型特征可视化 - 主图绘制完成
主图保存至: ./output/multi_epoch_visualizations/main_epoch_29.png
图例图保存至: ./output/multi_epoch_visualizations/legend_epoch_29.png
